In [1]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time

import scipy
import sksparse
from sksparse.cholmod import cholesky
from numba import jit

import scipy.sparse as sp


In [2]:
v, f = igl.read_triangle_mesh('data/bone_unify.obj')
labels = np.load('data/bone.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

In [3]:
handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ] 
    
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed) # p is a global variable, defined later
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

In [4]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value-1] #一个动画同步函数？
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

# My functions 1: for re-computation

In [5]:
# generate mask for deformation
def generate_mask(v, f, labels):
    hard_id = np.where(labels != 0)
    major_mask = np.ones(labels.shape[0], dtype=bool)
    major_mask[hard_id] = False
    constraint_mask = np.zeros(labels.shape[0], dtype=bool)
    constraint_mask[hard_id] = True
    return major_mask, constraint_mask

def generate_factor(W):
    LHS_op = -1 * W[major_mask][:,major_mask]
    factor = sksparse.cholmod.cholesky(LHS_op)
    return factor


def factor_RHS(W):
    R_op = []
    v_op = []
    
    for i in range(v_num):
        dataR = []
        iiR = []
        jjR = []
        datav = []
        iiv = []
        jjv = []
        count = 0
        
        for j in adj_list[i]:
            dataR.append(W[i,j]); iiR.append(count); jjR.append(i)
            dataR.append(W[i,j]); iiR.append(count); jjR.append(j)
            datav.append(1); iiv.append(count); jjv.append(i)
            datav.append(-1); iiv.append(count); jjv.append(j)
            count += 1
            
        R_op_i = sp.coo_matrix((dataR, (iiR, jjR)), shape=(count, v_num)).asformat("csr")
        v_op_i = sp.coo_matrix((datav, (iiv, jjv)), shape=(count, v_num)).asformat("csr")
        R_op.append(R_op_i)
        v_op.append(v_op_i)

    return R_op, v_op

# Pre-computation

In [6]:
# Precomputation
major_mask, constraint_mask = generate_mask(v,f,labels)
major_v = v[major_mask,:]
constraint_v = v[constraint_mask,:] 

adj_list = igl.adjacency_list(f)
v_num = v.shape[0]

cotM = igl.cotmatrix(v, f)
uniM = cotM.copy()
for a in range(v_num):
    uniM[a,a] = len(adj_list[a]) * -1
    for b in adj_list[a]:
        uniM[a,b] = 1
        
factor = generate_factor(cotM)
R_op, v_op = factor_RHS(cotM)

# My functions 2: for major iterations

In [7]:
def get_rotation(v_origin, v_target, W):
    R = np.zeros([v_num,3,3])
    for i in range(v_num):
        # 3 x #v
        P = (v_origin[adj_list[i]] - v_origin[i]).T
        # 3 x #v 
        P_d = (v_target[adj_list[i]] - v_target[i]).T
        D_array = W[i][:,adj_list[i]].toarray()
        D = np.diag(D_array[0])
        
        S = P @ D @ P_d.T
                
        U, Sigma, VT = np.linalg.svd(S, full_matrices=False)
        R[i] = VT.T@U.T
        
        if np.linalg.det(R[i]) < 0:
            R[i][:,-1] = -1 * R[i][:,-1]
    return R

def update_v(v_origin, v_target, W, R):
    LHS_op_0 = -1 * W
    
    RHS_0 = np.zeros([v_num, 3])
    for i in range(v_num):
        vector = np.zeros([3])
        v_diff = v_op[i]@v
        for j in range(len(adj_list[i])):
            vector += np.inner((R_op[i][j,i]*R[i]+R_op[i][j,j]), v_diff[j])
        RHS_0[i] = vector
        
    LHS_op = LHS_op_0[major_mask][:,major_mask]
    RHS = RHS_0[major_mask] - LHS_op_0[major_mask][:,constraint_mask]@v_target[constraint_mask]
    
    v_result = v_target.copy()
    v_result[major_mask] = factor(RHS)
    return v_result

@jit(forceobj=True, looplift=True)
def major_loop(v, v_move):
    error = 1
    count = 0
    v_current = v_move
    while((error>1e-3 and count <10)):
        count +=1
        
        R0 = get_rotation(v, v_current, cotM)        
        
        v_new = update_v(v, v_current, cotM, R0)
        
        error = np.linalg.norm(np.mean(v_new - v_current, axis = 0))
        v_current = v_new
        print(count, error)
    return v_new

# Realtime function

In [8]:
@jit(forceobj=True, looplift=True)
def position_deformer(target_pos):
    
    
    
    v_result = major_loop(v, target_pos)
    return v_result


pos_f.deformer = position_deformer


In [9]:
## Widget UI
# p = mp.plot(handle_vertex_positions, f, c=labels)
# iw.interact(pos_f,
#             **widgets_wrapper())

In [10]:
v_new, f_new = igl.read_triangle_mesh('data/bone_ditto.obj')

v_result = position_deformer(v_new)

mp.plot(v_result, c = labels, shading={"point_size": 0.1})

igl.write_triangle_mesh("data/bone_result_arap.obj", v_result, f_new)

1 0.0037606137365301597
2 0.0003462430443525275


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3522619…

True

In [11]:
mp.plot(v_result, f, c = labels, shading={"point_size": 0.1})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.3522619…